# Master 2 High Performance Computing and Simulation

## Members : lougani faouzi ,hery Andrianantenaina

## Report of module Data & Apprentissage TD2




## Introduction : 

This file contains our solutions for the different exercises that we have solved.
The work of these exercises has been shared between the two members of the group, so that we can discuss the problematic and seek together the solution of each exercise.

we used a git repository to version our progress: 
https://github.com/lougani-faouzi/DA

# Exercice 2.
**a)** Compare the performance of LOOCV with 5-fold CV.

For our tests we downloaded the sonar.csv file from: https://datahub.io/machine-learning/sonar#pandas
and we save it locally on the same folder of our work


In [3]:
# a. Comparaison of LOOCV and 5-fold CV : ---------------------------------
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import LeaveOneOut,cross_val_score
from sklearn.ensemble import RandomForestClassifier

# load dataset
url = './sonar.csv'
dataframe = read_csv(url, skiprows=[1])
data = dataframe.values

n_samples = len(data)
#print(n_samples)
# split into inputs and outputs
X, y = data[:, :-1], data[:, -1]

# create loocv procedure
cvLOOCV = LeaveOneOut()

# create model
model = RandomForestClassifier(random_state=1)

# evaluate model
loocv_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cvLOOCV, n_jobs=-1)
k_foldcv_scores = cross_val_score(model, X, y, scoring='accuracy', cv=5, n_jobs=-1)

# report performance
print('Perfor report  :')
print('The accuracy for LOOCV     : %.3f (%.3f)' % (mean(loocv_scores), std(loocv_scores)))
print('The accuracy for 5-fold CV : %.3f (%.3f)' % (mean(k_foldcv_scores), std(k_foldcv_scores)))

Perfor report  :
The accuracy for LOOCV     : 0.836 (0.371)
The accuracy for 5-fold CV : 0.716 (0.141)
